In [30]:
import os
import requests
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import json
import numpy as np



In [31]:
def extraccion_actual(fecha, CLIENT_ID, manana):

    url="https://api.seatgeek.com/2/events"
    params = {
    "client_id": CLIENT_ID,
    "venue.city": "New York",   
    "sort": "score.desc",        
    "per_page": 100,            
    "datetime_local.gte": fecha,
    "datetime_local.lte": manana,
    }
    
    response = requests.get(url, params=params)
    assert response.status_code == 200, "Error en la extracción de eventos"

    return response.json()

    
    

In [32]:
fecha_hoy_obj = datetime.now() 
manana_obj = fecha_hoy_obj + timedelta(days = 1)

fecha_hoy_str = fecha_hoy_obj.strftime('%Y-%m-%d')
manana_str = manana_obj.strftime('%Y-%m-%d')


API_KEY = os.getenv('CLIENT_ID_SEATGEEK')
assert API_KEY is not None, "Falta la variable de entorno CLIENT_ID_SEATGEEK"


data = extraccion_actual(fecha_hoy_str, API_KEY, manana_str)


In [33]:
data

{'events': [{'access_method': None,
   'announce_date': '2025-08-01T00:00:00',
   'announcements': {},
   'conditional': False,
   'contingent': False,
   'created_at': '2025-08-01T14:28:59',
   'date_tbd': False,
   'datetime_local': '2026-02-17T19:00:00',
   'datetime_tbd': False,
   'datetime_utc': '2026-02-18T00:00:00',
   'description': '',
   'onsale': None,
   'enddatetime_utc': '2026-02-18T00:00:00',
   'event_promotion': None,
   'game_number': 0,
   'home_game_number': 0,
   'id': 17651273,
   'integrated': None,
   'is_open': False,
   'is_visible': True,
   'is_visible_override': 'UNSET',
   'links': [{'id': '159062876',
     'url': '',
     'provider': 'stubhub',
     'primary': False,
     'display_name': '',
     'logos': {},
     'sales': [],
     'link_type': 'ID'},
    {'id': '3471153',
     'url': '',
     'provider': 'rapidseats',
     'primary': False,
     'display_name': '',
     'logos': {},
     'sales': [],
     'link_type': 'ID'},
    {'id': 'Z1r9uZrrZbpZ1Avq

In [34]:
eventos_limpios = []

for e in data['events']:
    info = {
        'nombre_evento': e.get('title'),
        'tipo': e.get('type'),
        'hora_inicio': e.get('datetime_local'),
        'lugar': e['venue'].get('name'),
        'direccion': e['venue'].get('address', 'Dirección no disponible'),
        'latitud': e['venue']['location'].get('lat'),
        'longitud': e['venue']['location'].get('lon'),
        'capacidad': e['venue'].get('capacity'),
        'popularidad_score': e.get('score'), 
        'venue_score': e['venue'].get('score') 
    }
    eventos_limpios.append(info)

df = pd.DataFrame(eventos_limpios)

In [35]:
df['capacidad'] = df['capacidad'].replace(0, np.nan)
#df = df.drop(["latitud", "longitud"], axis= 1)

In [36]:
df['hora_inicio'] = pd.to_datetime((df['hora_inicio']))
df['hora_inicio'] = df['hora_inicio'].dt.strftime('%H:%M')

In [37]:
tiempos_salida = {
    
    'nba': 2.5, 'nfl': 3.5, 'mlb': 3.0, 'nhl': 2.5, 'mls': 2.0, 
    'ncaa_basketball': 2.5, 'ncaa_football': 3.5, 'sports': 2.5,
    'tennis': 4.0, 'wwe': 3.0, 'boxing': 3.5, 'mma': 3.5,
    
    
    'concert': 3.0, 'music_festival': 8.0, 'classical': 2.5, 'opera': 3.0,
    
    
    'theater': 2.5, 'broadway_tickets_national': 2.5, 'comedy': 2.0, 
    'family': 2.0, 'ballet': 2.5, 'cirque_du_soleil': 2.0
}

In [38]:
def calcular_salida(fila):
    tipo_evento = fila['tipo']
    horas_duracion = tiempos_salida.get(tipo_evento, 2.5) 
    hora_inicio = pd.to_datetime(fila['hora_inicio'])
    hora_fin = hora_inicio + timedelta(hours=horas_duracion)
    
    return hora_fin.strftime('%H:%M')

In [39]:
df['hora_salida_estimada'] = df.apply(calcular_salida, axis=1)


In [40]:
df["coordinates"] = df.apply(lambda fila: [fila['longitud'], fila['latitud']], axis=1)
df = df.drop(['longitud', 'latitud', 'lugar', 'direccion', 'tipo'], axis = 1)

In [41]:
df

,nombre_evento,hora_inicio,capacidad,popularidad_score,venue_score,hora_salida_estimada,coordinates
0,Hamilton - New York,19:00,NaN,0.489783,0.79,21:30,"[-73.9866, 40.7592]"
1,Hell's Kitchen - New York,19:00,NaN,0.489270,0.65,21:30,"[-73.9875, 40.7579]"
2,Chicago - The Musical - New York,19:00,1155.0,0.478968,0.69,21:30,"[-73.985, 40.7613]"
3,Romy & Michele: The Musical - New York,19:00,NaN,0.429742,0.50,21:30,"[-73.9939, 40.7587]"
4,Wicked - New York,19:00,NaN,0.464753,0.76,21:30,"[-73.9851, 40.7624]"
5,The Outsiders - New York,19:00,NaN,0.459291,0.72,21:30,"[-73.9878, 40.7586]"
6,Just In Time - New York,19:00,NaN,0.456763,0.74,21:30,"[-73.9852, 40.7623]"
7,Death Becomes Her - New York,19:00,NaN,0.452506,0.69,21:30,"[-73.986, 40.759]"
8,Bug - New York,19:00,NaN,0.451224,0.59,21:30,"[-73.9868, 40.7602]"
9,Six the Musical - New York,19:00,NaN,0.392834,0.68,21:30,"[-73.9868, 40.7602]"
